<a href="https://colab.research.google.com/github/SuYouge/colab/blob/master/reuse_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 加载模型

## 导入tf2.0

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

## 挂载云盘

输出栏会显示一个`Go to this URL in a browser:xxx`
点击链接跳转后按照指示复制验证码到输出栏的输入框中即可

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/mnist_test"
os.chdir(path)
os.listdir(path)

## 加载模型

In [0]:
# 重新创建完全相同的模型，包括其权重和优化程序
model = tf.keras.models.load_model('my_model.h5')
# 显示网络结构
model.summary()

## 进一步训练

没有改变以下选项则不需要重新`compile`：损失函数、优化器 / 学习率、度量

In [0]:
# 重新导入数据集

def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0 # 将MNIST数据映射到[0，1]
    x = tf.expand_dims(x, axis=-1) # 由于卷积层维度为[None, 28, 28, 1]，故在axis=3扩展一维
    # y = tf.one_hot(y, depth=10)
    return x, y
def load_dataset(mnist):

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  print(x_train.shape, y_train.shape, x_train.min(), x_train.max())

  train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  train_db = train_db.shuffle(1000).map(preprocess).batch(100)

  test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
  test_db = test_db.map(preprocess).batch(100)

  return train_db, test_db
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train, y_train = preprocess(x_train,y_train)
x_test, y_test = preprocess(x_test,y_test)
train_db, test_db= load_dataset(mnist)

# 重新配置训练流程

filepath = 'my_model.h5' # 保存模型地址
saved_model = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 2) # 回调保存模型功能
tensorboard = tf.keras.callbacks.TensorBoard(log_dir = 'log') # 回调可视化数据功能

# 开始训练

# model.fit(x_train, y_train, epochs=5)
history = model.fit(train_db, 
            epochs = 25, 
            validation_data = test_db, 
            validation_freq = 1,
            callbacks = [saved_model, tensorboard],
            verbose = 2)
print("\n")

# 显示训练记录
history.history